In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE236924"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE236924"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE236924.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE236924.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE236924.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"SIRPa agonist antibody treatment ameliorates experimental arthritis and colitis [array]"
!Series_summary	"The innate immune system is finely tuned to enable. rapid response to pathogenic stimuli but keep quiescent during tissue homeostasis. Balance of activating and inhibitory signaling sets a threshold for immune activation. Signal regulatory protein (SIRPa) is an immune inhibitory receptor expressed by myeloid cells and interacts with CD47 to inhibit immune cell phagocytosis, migration, and activation. Despite the progress of SIRPa and CD47 antagonist antibodies to promote anti-cancer immunity, it is not yet known whether therapeutic SIRPa receptor agonism could restrain excessive autoimmune inflammation in the context of autoimmunity. Here, we reported that increased neutrophil- and monocyte-associated genes including SIRPA in inflamed tissues biopsies of rheumatoid arthritis and inflammatory bowel diseases, and elevated SIRPA in colonic biopsi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Let's analyze the given information

# 1. Gene Expression Data Availability
# Based on the title and summary, this appears to be an array dataset comparing joint tissues
# from RA, OA and control samples. This suggests it contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary, we only have one key (0) with disease information
trait_row = 0  # Disease status is available in row 0
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait values to binary (0 for Control/OA, 1 for RA)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary where RA=1, others=0
    if value == "RA":
        return 1
    elif value in ["Control", "OA"]:
        return 0
    else:
        return None

# No need for age and gender conversion functions since data is not available

# 3. Save Metadata - Initial Filtering
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=None,
        gender_row=gender_row,
        convert_gender=None
    )
    
    # Preview the extracted clinical data
    print("Preview of clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Save clinical data to file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM7585682': [0.0], 'GSM7585683': [0.0], 'GSM7585684': [0.0], 'GSM7585685': [0.0], 'GSM7585686': [1.0], 'GSM7585687': [0.0], 'GSM7585688': [0.0], 'GSM7585689': [0.0], 'GSM7585690': [0.0], 'GSM7585691': [1.0], 'GSM7585692': [0.0], 'GSM7585693': [1.0], 'GSM7585694': [0.0], 'GSM7585695': [0.0], 'GSM7585696': [0.0], 'GSM7585697': [1.0], 'GSM7585698': [1.0], 'GSM7585699': [0.0], 'GSM7585700': [0.0], 'GSM7585701': [1.0], 'GSM7585702': [0.0], 'GSM7585703': [0.0], 'GSM7585704': [0.0], 'GSM7585705': [0.0], 'GSM7585706': [0.0], 'GSM7585707': [0.0], 'GSM7585708': [1.0], 'GSM7585709': [1.0], 'GSM7585710': [0.0], 'GSM7585711': [0.0], 'GSM7585712': [0.0], 'GSM7585713': [1.0], 'GSM7585714': [0.0], 'GSM7585715': [0.0], 'GSM7585716': [1.0], 'GSM7585717': [0.0], 'GSM7585718': [0.0], 'GSM7585719': [0.0], 'GSM7585720': [0.0], 'GSM7585721': [0.0], 'GSM7585722': [1.0], 'GSM7585723': [1.0], 'GSM7585724': [0.0], 'GSM7585725': [0.0], 'GSM7585726': [0.0], 'GSM7585727': [0.0], 'GSM758

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers in the gene expression data don't appear to be human gene symbols
# They look like Affymetrix probe IDs (like "1007_s_at", "1053_at") which need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. From the gene annotation preview, we can see:
# - 'ID' contains probe identifiers matching those in gene_data.index
# - 'Gene Symbol' contains the gene symbols we want to map to

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print a sample of the mapping to verify
print("Sample of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the result and a sample of gene symbols to verify
print("\nAfter mapping, gene expression data shape:", gene_data.shape)
print("Sample of gene symbols in the mapped data:")
print(gene_data.index[:10])


Sample of gene mapping:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A



After mapping, gene expression data shape: (21278, 132)
Sample of gene symbols in the mapped data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("Sample of normalized gene symbols:")
print(normalized_gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# First, let's reload the clinical data to ensure we have the correct structure
clinical_data_reloaded = geo_select_clinical_features(
    clinical_data, 
    trait, 
    trait_row, 
    convert_trait,
    None,  # age_row is None 
    None,  # convert_age is None
    None,  # gender_row is None
    None   # convert_gender is None
)

# Print column names to debug
print("Clinical data index:", clinical_data_reloaded.index)

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_reloaded, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data column names:", linked_data.columns[:10])  # Print some column names

# 4. Handle missing values in the linked data
# The first column should contain the trait data
trait_column_name = clinical_data_reloaded.index[0]
print(f"Using trait column name: {trait_column_name}")
linked_data = handle_missing_values(linked_data, trait_column_name)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait_column_name)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data for Rheumatoid Arthritis cases versus controls (OA and normal samples)."
)

# 7. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Usable linked data saved to {out_data_file}")
else:
    print("Linked data was not usable and was not saved.")

Normalized gene data shape: (19845, 132)
Sample of normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE236924.csv
Clinical data index: Index(['Rheumatoid_Arthritis'], dtype='object')
Linked data shape: (132, 19846)
Linked data column names: Index(['Rheumatoid_Arthritis', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1',
       'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT'],
      dtype='object')
Using trait column name: Rheumatoid_Arthritis


Linked data shape after handling missing values: (132, 19846)
For the feature 'Rheumatoid_Arthritis', the least common label is '1.0' with 36 occurrences. This represents 27.27% of the dataset.
The distribution of the feature 'Rheumatoid_Arthritis' in this dataset is fine.



Usable linked data saved to ../../output/preprocess/Rheumatoid_Arthritis/GSE236924.csv
